In [1]:
install.packages(c("cowplot",
"digest",
"infer",
"repr",
"tidyverse",
"dplyr"))

Installing packages into ‘/home/kolcr/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rappdirs’, ‘data.table’, ‘cli’, ‘gargle’, ‘ids’, ‘rematch2’, ‘rlang’, ‘ellipsis’, ‘vctrs’, ‘dtplyr’, ‘googledrive’, ‘googlesheets4’, ‘pillar’




In [5]:
cleveland <- read.csv('https://raw.githubusercontent.com/KolCrooks/STAT201Project/master/clevelanddata_mod.csv')

head(cleveland)

,prop,max.hr,resting.hr,max.restinghr,peakbps,peakbpd,restbpd,max.restingbpd,X,prop.1,⋯,peakbpd.1,restbpd.1,max.restingbpd.1,X.1,X.2,Max.HR,Max...Resting.HR,Peak.BPS,Peak.BPD,Max...Resting.Average.BP
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<lgl>,<int>,⋯,<int>,<int>,<dbl>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,150,60,90,190,90,85,55.0,NA,1,⋯,90,90,35.0,NA,t-test,1.059170e-07,1.536030e-30,0.07091353,0.1699155,3.085300e-25
2,0,172,71,101,160,74,86,31.0,NA,1,⋯,80,80,30.0,NA,no prop,1.552527e+02,7.717742e+01,169.98924730,78.0268817,3.921774e+01
3,0,178,73,105,165,70,75,42.5,NA,1,⋯,68,78,53.5,NA,prop,1.391170e+02,3.539785e+01,164.46808510,80.2978723,1.872043e+01
4,0,160,83,77,180,84,100,32.0,NA,1,⋯,70,105,-10.0,NA,,NA,NA,NA,NA,NA
5,0,163,84,79,165,80,80,42.5,NA,1,⋯,120,100,52.5,NA,,NA,NA,NA,NA,NA
6,0,148,86,62,180,75,90,37.5,NA,1,⋯,118,100,61.0,NA,,NA,NA,NA,NA,NA


# Some grand title

- 500 words (about 1 page)
- identifies the dataset they plan to work on
- question they would like to answer using that dataset for their group project.
- in a Jupyter notebook, and then submitted both as an .html file (File → Download As → HTML) and an .ipynb file that is reproducible (i.e. works and runs without any additional files).

Only one member of your team needs to submit. 
## Introduction

- provide some relevant background information on the topic so that someone unfamiliar with it will understand your proposal.
- Clearly state the question you will try to answer with your project.
- One random variable of interest, spread across two or more categories that are interesting to compare.
 - examples, consider the annual maxima river flow at two different locations along a river
 - gender diversity at different universities.
- Of the response variable, identify one location parameter (mean, median, quantile, etc.) and one scale parameter (standard deviation, inter-quartile range, etc.) that would be useful in answering your question.
- Justify your choices.
- Identify and describe the dataset that will be used to answer the question.
- this dataset is allowed to contain more variables than you need – feel free to drop them!
- Also, be sure to frame your question/objectives in terms of what is already known in the literature.
- Be sure to include at least two scientific publications that can help frame your study (you will need to include these in the References section).

---You must submit two files:

- the source Jupyter notebook (.ipynb file)

- the rendered final document (.html file)

---


## Introduction

- provide some relevant background information on the topic so that someone unfamiliar with it will understand your proposal.
- Clearly state the question you will try to answer with your project.
- One random variable of interest, spread across two or more categories that are interesting to compare.
 - examples, consider the annual maxima river flow at two different locations along a river
 - gender diversity at different universities.
- Of the response variable, identify one location parameter (mean, median, quantile, etc.) and one scale parameter (standard deviation, inter-quartile range, etc.) that would be useful in answering your question.
- Justify your choices.
- Identify and describe the dataset that will be used to answer the question.
- this dataset is allowed to contain more variables than you need – feel free to drop them!
- Also, be sure to frame your question/objectives in terms of what is already known in the literature.
- Be sure to include at least two scientific publications that can help frame your study (you will need to include these in the References section).

---

## Preliminary Results

- Demonstrate that the dataset can be read from the web into R.
- Clean and wrangle your data into a tidy format.
- Plot the relevant raw data, tailoring your plot in a way that
- addresses your question.
- Compute estimates of the parameter you identified across your groups. Present this in a table. If relevant, include these estimates in your plot.
- Be sure to not print output that takes up a lot of screen space.

---



## Methods: Plan

The previous sections will carry over to your final report (you’ll be allowed to improve them based on feedback you get). Begin this Methods section with a brief description of “the good things” about this report – specifically, in what ways is this report trustworthy?

Continue by explaining why the plot(s) and estimates that you produced are not enough to give to a stakeholder, and what you should provide in addition to address this gap. Make sure your plans include at least one hypothesis test and one confidence interval, with at least one of these being calculated using both bootstrapping and asymptotics for comparison.

Finish this section by reflecting on how your final report might play out:

What do you expect to find?

What impact could such findings have?

What future questions could this lead to?

---


## References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.